**Preprocessing!**

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import numpy
import nltk
import sys
import os
import re
import string

In [2]:
import numpy as np
import pandas as pd
import os
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
reviews = pd.read_csv('data/deceptive-opinion.csv')

In [4]:
reviews["text_length"] = reviews["text"].map(str).apply(len)

In [5]:
def tokenizeText(text):
    return nltk.word_tokenize(text)

def sortTokensByFreq(tokensList):
    word_frequencies = nltk.FreqDist(tokensList)
    return sorted(word_frequencies.items() , key = lambda x: x[1] ,
                       reverse = True)

def tokenizeAndSort(text):
    tok = tokenizeText(text)
    return sortTokensByFreq(tok)
def toLowerCase(text):
    return text.lower()

def removePunctuation(text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub(" ", text)

def removeNumbers(text):
    #·text = re.sub(pt.get_number_pattern(), r' ', text)
    text = re.sub("[0-9]+", " ", text)
    return text    

def removeStopWords(text):    
    pattern = re.compile(r'\b(' + r'|'.join(nltk.corpus.stopwords.words("english")) + r')\b\s*')
    text = pattern.sub(" ", text)
    return text

def replaceExtraWhitespaces(text):
    pattern = re.compile(r"[ \n\t]+")
    text = pattern.sub(" ", text)   
    # Strip leading whitespaces.
    text = text.strip()
    return text

def normalize(text):
    text = toLowerCase(text)
    text = removePunctuation(text)
    text = removeNumbers(text)
    text = removeStopWords(text)
    text = replaceExtraWhitespaces(text)
    return text

https://simonhessner.de/lemmatize-whole-sentences-with-python-and-nltks-wordnetlemmatizer/

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:          
    return None
def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:            
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))
  return " ".join(res_words)

In [7]:
def stemmingWords_withPorter(text):
    ps = nltk.stem.PorterStemmer()
    text = " ".join([ps.stem(w) for w in nltk.word_tokenize(text)])
    return text


nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

In [8]:
reviews["lower_case"] = reviews["text"].apply(toLowerCase)
reviews["lc_no_punct"] = reviews["lower_case"].apply(removePunctuation)
reviews["norm"] = reviews["lc_no_punct"].apply(removePunctuation)

reviews["norm_lemma"] = reviews["norm"].apply(lemmatize_sentence)
reviews["norm_stem"] = reviews["norm"].apply(stemmingWords_withPorter)

reviews["norm_lemma_stopword"] = reviews["norm_lemma"].apply(removeStopWords)
reviews["norm_stem_stopword"] = reviews["norm_stem"].apply(removeStopWords)

In [9]:
reviews.head()

,deceptive,hotel,polarity,source,text,text_length,lower_case,lc_no_punct,norm,norm_lemma,norm_stem,norm_lemma_stopword,norm_stem_stopword
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...,572,we stayed for a one night getaway with family ...,we stayed for a one night getaway with family ...,we stayed for a one night getaway with family ...,we stay for a one night getaway with family on...,we stay for a one night getaway with famili on...,stay one night getaway family thursday t...,stay one night getaway famili thursday t...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...,286,triple a rate with upgrade to view room was le...,triple a rate with upgrade to view room was le...,triple a rate with upgrade to view room was le...,triple a rate with upgrade to view room be les...,tripl a rate with upgrad to view room wa less ...,triple rate upgrade view room less 200 a...,tripl rate upgrad view room wa less 200 a...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...,1104,this comes a little late as i'm finally catchi...,this comes a little late as i m finally catchi...,this comes a little late as i m finally catchi...,this come a little late as i m finally catch u...,thi come a littl late as i m final catch up on...,come little late finally catch review ...,thi come littl late final catch review ...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...,707,the omni chicago really delivers on all fronts...,the omni chicago really delivers on all fronts...,the omni chicago really delivers on all fronts...,the omni chicago really deliver on all front f...,the omni chicago realli deliv on all front fro...,omni chicago really deliver front spaciou...,omni chicago realli deliv front spacious ...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...,384,i asked for a high floor away from the elevato...,i asked for a high floor away from the elevato...,i asked for a high floor away from the elevato...,i ask for a high floor away from the elevator ...,i ask for a high floor away from the elev and ...,ask high floor away elevator get ro...,ask high floor away elev got room w...


In [10]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Susan\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [11]:
from nltk import word_tokenize, pos_tag

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        tokens = word_tokenize(x)
        for tup in pos_tag(tokens, tagset='universal'):
            ppo = list(tup)[1]
            if ppo == flag:
                cnt += 1
    except:
        pass
    return cnt / len(tokens)


In [12]:
pos_list = [
  "ADJ",
  "ADP",
  "ADV",
  "AUX",
  "CCONJ",
  "DET",
  "INTJ",
  "NOUN",
  "NUM",
  "PART",
  "PRON",
  "PROPN",
  "PUNCT",
  "SCONJ",
  "SYM",
  "VERB",
  "X"
]

In [13]:
for pos in pos_list:
  reviews[pos + "_count"] = reviews['lc_no_punct'].apply(lambda x: check_pos_tag(x, pos))

In [14]:
pd.set_option('display.max_columns', 500)

In [15]:
reviews.head()

,deceptive,hotel,polarity,source,text,text_length,lower_case,lc_no_punct,norm,norm_lemma,norm_stem,norm_lemma_stopword,norm_stem_stopword,ADJ_count,ADP_count,ADV_count,AUX_count,CCONJ_count,DET_count,INTJ_count,NOUN_count,NUM_count,PART_count,PRON_count,PROPN_count,PUNCT_count,SCONJ_count,SYM_count,VERB_count,X_count
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...,572,we stayed for a one night getaway with family ...,we stayed for a one night getaway with family ...,we stayed for a one night getaway with family ...,we stay for a one night getaway with family on...,we stay for a one night getaway with famili on...,stay one night getaway family thursday t...,stay one night getaway famili thursday t...,0.148148,0.120370,0.074074,0.0,0.0,0.101852,0.0,0.277778,0.064815,0.0,0.037037,0.0,0.0,0.0,0.0,0.129630,0.0
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...,286,triple a rate with upgrade to view room was le...,triple a rate with upgrade to view room was le...,triple a rate with upgrade to view room was le...,triple a rate with upgrade to view room be les...,tripl a rate with upgrad to view room wa less ...,triple rate upgrade view room less 200 a...,tripl rate upgrad view room wa less 200 a...,0.136364,0.113636,0.068182,0.0,0.0,0.090909,0.0,0.318182,0.022727,0.0,0.000000,0.0,0.0,0.0,0.0,0.227273,0.0
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...,1104,this comes a little late as i'm finally catchi...,this comes a little late as i m finally catchi...,this comes a little late as i m finally catchi...,this come a little late as i m finally catch u...,thi come a littl late as i m final catch up on...,come little late finally catch review ...,thi come littl late final catch review ...,0.137441,0.151659,0.066351,0.0,0.0,0.118483,0.0,0.227488,0.014218,0.0,0.061611,0.0,0.0,0.0,0.0,0.132701,0.0
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...,707,the omni chicago really delivers on all fronts...,the omni chicago really delivers on all fronts...,the omni chicago really delivers on all fronts...,the omni chicago really deliver on all front f...,the omni chicago realli deliv on all front fro...,omni chicago really deliver front spaciou...,omni chicago realli deliv front spacious ...,0.062500,0.156250,0.023438,0.0,0.0,0.179688,0.0,0.335938,0.023438,0.0,0.015625,0.0,0.0,0.0,0.0,0.140625,0.0
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...,384,i asked for a high floor away from the elevato...,i asked for a high floor away from the elevato...,i asked for a high floor away from the elevato...,i ask for a high floor away from the elevator ...,i ask for a high floor away from the elev and ...,ask high floor away elevator get ro...,ask high floor away elev got room w...,0.123288,0.095890,0.054795,0.0,0.0,0.123288,0.0,0.273973,0.013699,0.0,0.041096,0.0,0.0,0.0,0.0,0.205479,0.0


In [16]:
reviews["class"] = 0
reviews["class"] = np.where(((reviews['class'] == 0) & (reviews['deceptive'] == 'truthful') & (reviews['polarity'] == 'positive')) , 1, 0)
reviews["class"] = np.where(((reviews['class'] == 0) & (reviews['deceptive'] == 'truthful') & (reviews['polarity'] == 'negative')) , 2, reviews["class"])
reviews["class"] = np.where(((reviews['class'] == 0) & (reviews['deceptive'] == 'deceptive') & (reviews['polarity'] == 'positive')) , 3, reviews["class"])
reviews["class"] = np.where(((reviews['class'] == 0) & (reviews['deceptive'] == 'deceptive') & (reviews['polarity'] == 'negative')) , 4, reviews["class"])

In [17]:
reviews["deceptive"] = np.where(((reviews['deceptive'] == "deceptive")) , 1, 0)
reviews["polarity"] = np.where(((reviews['polarity'] == "positive")) , 1, 0)


In [18]:
reviews.to_csv("data/reviews_mod5.csv")